In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [94]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [95]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [96]:
items_weights = data_train.groupby(by='item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value':'weight'})

In [97]:
items_weights['weight'] += 1
items_weights['weight'] = np.log(items_weights['weight'])
items_weights['weight'] /= items_weights['weight'].sum()
items_weights['weight'].sum()

1.0

In [98]:
def weighted_random_recommendation(items_weights, n=5):
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    return recs.tolist()

In [99]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [100]:
%%time
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

Wall time: 4.22 s


In [101]:
result.head(3)

,user_id,actual,weighted_random_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[13381256, 12673453, 1074209, 9802691, 831557]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [102]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [103]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 7.96 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]"


In [104]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [106]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 221 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [107]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [145]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [147]:
top_5000

[6534178,
 6533889,
 6534166,
 6544236,
 1404121,
 397896,
 999999,
 1426702,
 5703832,
 480014,
 5668996,
 6410462,
 420647,
 5747420,
 731106,
 5716076,
 707683,
 202291,
 1153346,
 1082185,
 5747233,
 6410464,
 545926,
 995242,
 2848087,
 1388206,
 1029743,
 5712216,
 5850988,
 1133018,
 1106523,
 1007195,
 981760,
 5845857,
 883404,
 1127831,
 2690723,
 866227,
 995785,
 860776,
 951590,
 5569230,
 908531,
 916122,
 826249,
 1098066,
 862349,
 1058997,
 1044078,
 904360,
 840361,
 923746,
 1126899,
 849843,
 961554,
 1005186,
 820165,
 1053690,
 844179,
 844165,
 1070820,
 1065593,
 834484,
 994928,
 833715,
 859075,
 1022003,
 1013321,
 938700,
 854852,
 986947,
 5569471,
 927191,
 1071939,
 1096036,
 1004906,
 986912,
 1080414,
 914190,
 908846,
 962229,
 8090521,
 1085604,
 911878,
 1092026,
 866211,
 1068719,
 878996,
 8090537,
 1081177,
 847270,
 929668,
 1024306,
 909894,
 907014,
 903325,
 910032,
 1095275,
 833025,
 962568,
 862139,
 953476,
 847982,
 999971,
 1028816,
 968

In [112]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [113]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [114]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N-1
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.85 s


In [115]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 197 ms


In [116]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]"


In [117]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.76 s


In [118]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 140 ms


In [119]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 999999, 981760, 1098066, 1127831]"


In [123]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.91 s


In [124]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 156 ms


In [125]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 999999, 981760, 1098066, 1127831]","[1082185, 981760, 999999, 1127831, 1098066]"


In [126]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.93 s


In [127]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

Wall time: 117 ms


In [128]:
result.head(5)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 999999, 981760, 1098066, 1127831]","[1082185, 981760, 999999, 1127831, 1098066]","[1082185, 1098066, 1127831, 1004906, 1081177]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[13381256, 12673453, 1074209, 9802691, 831557]","[15595912, 13017642, 6514251, 12673048, 12484608]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[861792, 8198836, 1061975, 862753, 842330]","[1823518, 6773231, 312652, 822944, 6944764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1029743, 1044078, 5569230, 1076056, 9527290]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[262437, 16056721, 937881, 1024998, 8012554]","[969403, 6464150, 1050422, 7144131, 2674370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 961554, 1098066]","[1082185, 981760, 961554, 999999, 1098066]","[1029743, 1096036, 5568378, 1071939, 1126899]"


In [130]:
result.to_csv('predictions_basic_2.csv', index=False)

In [131]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

for column_name in result.columns[1:]:
    print('%s: %f' % (column_name, result.apply(lambda row: precision_at_k(row[column_name], row['actual']), axis=1).mean()))

actual: 1.000000
weighted_random_recommendation: 0.012965
random_recommendation: 0.006883
popular_recommendation: 0.393037
itemitem: 0.543737
cosine: 0.575830
tfidf: 0.608723


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


own_purchases: 1.000000


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [132]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [133]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
items = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [169]:
res = result.apply(lambda x: precision_at_k(x['random_recommendation_5000'], x['actual'],  5), axis=1)
metric = pd.DataFrame(index=['precision@5'])
metric['random_recommendation_5000'] = res.mean();

In [170]:
metric[['random_recommendation_5000']]

,random_recommendation_5000
precision@5,0.055862


Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾 .

In [139]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [143]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


In [151]:
user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [156]:
result_test = result[['user_id','actual']]

In [157]:
k_list = np.arange(1, 100, 2)

In [161]:
k_list

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67,
       69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99])

In [ ]:
%%time
for k in k_list:
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    
    #result_test - и дальше не понимаю как рправильно написать код.....

In [162]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.62 s


In [163]:
%%time

result['new_own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

Wall time: 126 ms


In [164]:
result.head(5)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5000,new_own_purchases
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[892334, 1148599, 1106592, 838563, 2186629]","[64714, 1039529, 5567592, 123983, 6772852]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[953539, 828525, 904833, 1040553, 15452677]","[1082185, 981760, 995242, 1029743, 840361]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[12757617, 5995172, 12128635, 7123871, 12487293]","[1925691, 9420144, 1527524, 13190028, 1019108]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 999999, 981760, 1098066, 1127831]","[1082185, 981760, 999999, 1127831, 1098066]","[1082185, 1098066, 1127831, 1004906, 1081177]","[1116376, 912004, 7168417, 6773204, 1093013]","[1082185, 981760, 840361, 1098066, 826249]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[13381256, 12673453, 1074209, 9802691, 831557]","[15595912, 13017642, 6514251, 12673048, 12484608]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[5845857, 13115493, 9707199, 919589, 931255]","[1082185, 981760, 1098066, 6534178, 826249]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[861792, 8198836, 1061975, 862753, 842330]","[1823518, 6773231, 312652, 822944, 6944764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1029743, 1044078, 5569230, 1076056, 9527290]","[839094, 868319, 826790, 1119112, 1127716]","[1082185, 981760, 1029743, 840361, 883404]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[262437, 16056721, 937881, 1024998, 8012554]","[969403, 6464150, 1050422, 7144131, 2674370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 961554, 1098066]","[1082185, 981760, 961554, 999999, 1098066]","[1029743, 1096036, 5568378, 1071939, 1126899]","[9420336, 865196, 891141, 1097815, 1128900]","[1082185, 981760, 1029743, 961554, 916122]"


In [165]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

for column_name in result.columns[1:]:
    print('%s: %f' % (column_name, result.apply(lambda row: precision_at_k(row[column_name], row['actual']), axis=1).mean()))

actual: 1.000000
weighted_random_recommendation: 0.012965
random_recommendation: 0.006883
popular_recommendation: 0.393037
itemitem: 0.543737
cosine: 0.575830
tfidf: 0.608723
own_purchases: 1.000000
random_recommendation_5000: 0.055862
new_own_purchases: 0.881753
